In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [28]:
CLIENT_ID = 'Y3VXY10YN00C5HLQEPM3GZL4JLGZRN31YB1IAIBXYRDMORVQ' # your Foursquare ID
CLIENT_SECRET = 'D3LF5XOY4SGC0NHAVT3UWKUO55HK14GF5MVL2DXIXJKL0MST' # your Foursquare Secret
VERSION = '20181005' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y3VXY10YN00C5HLQEPM3GZL4JLGZRN31YB1IAIBXYRDMORVQ
CLIENT_SECRET:D3LF5XOY4SGC0NHAVT3UWKUO55HK14GF5MVL2DXIXJKL0MST


In [29]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [30]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [31]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=Y3VXY10YN00C5HLQEPM3GZL4JLGZRN31YB1IAIBXYRDMORVQ&client_secret=D3LF5XOY4SGC0NHAVT3UWKUO55HK14GF5MVL2DXIXJKL0MST&v=20181005&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb7341fdb04f55c42dfc912'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin' Donuts,Donut Shop,40.876993,-73.906507


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [37]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,94,94,94,94,94,94
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


In [38]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Re

In [ ]:
manhattan_onehot.head()

In [48]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Re

In [ ]:
manhattan_grouped.shape

In [40]:
num_top_venues = 50

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                    venue  freq
0                    Park  0.07
1             Coffee Shop  0.07
2                   Hotel  0.05
3      Italian Restaurant  0.03
4               Wine Shop  0.03
5                     Gym  0.03
6                Fountain  0.02
7           Shopping Mall  0.02
8            Cupcake Shop  0.02
9               BBQ Joint  0.02
10         Ice Cream Shop  0.02
11       Department Store  0.02
12         Sandwich Place  0.02
13           Burger Joint  0.02
14          Boat or Ferry  0.02
15          Memorial Site  0.02
16                  Plaza  0.02
17    American Restaurant  0.02
18                 Market  0.02
19              Gift Shop  0.01
20     Chinese Restaurant  0.01
21             Steakhouse  0.01
22            Candy Store  0.01
23          Grocery Store  0.01
24           Gourmet Shop  0.01
25          Historic Site  0.01
26          Burrito Place  0.01
27  Performing Arts Venue  0.01
28       Sushi Restaurant  0.01
29            

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Wine Shop,Gym,Market,Shopping Mall,Boat or Ferry,Sandwich Place,Plaza,Cupcake Shop,Department Store,BBQ Joint,Memorial Site,Fountain,Burger Joint,Ice Cream Shop,American Restaurant,Garden,Lingerie Store,Burrito Place,Smoke Shop,Candy Store,Sculpture Garden,Indian Restaurant,Scenic Lookout,Women's Store,Chinese Restaurant,Public Art,Historic Site,Cosmetics Shop,Grocery Store,Playground,Pizza Place,Gourmet Shop,Pet Store,Performing Arts Venue,Juice Bar,Gift Shop,Dog Run,Electronics Store,Building,Gastropub,Monument / Landmark,Mexican Restaurant,Food Court,Food Truck,Movie Theater,Pub
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Spa,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore,Bar,Wine Shop,Thai Restaurant,Cocktail Bar,Grocery Store,Gym / Fitness Center,Bakery,Art Museum,Sports Bar,Italian Restaurant,Hot Dog Joint,Pub,Nail Salon,Vietnamese Restaurant,Playground,Clothing Store,Karaoke Bar,Museum,Kosher Restaurant,Chinese Restaurant,Food Truck,Flower Shop,Bagel Shop,Ramen Restaurant,Shoe Store,Filipino Restaurant,Breakfast Spot,Exhibit,Burger Joint,Mediterranean Restaurant,Salon / Barbershop,Restaurant,Wine Bar,Mexican Restaurant,Women's Store,Ice Cream Shop,Dance Studio,American Restaurant,Vegetarian / Vegan Restaurant,Concert Hall
2,Central Harlem,Chinese Restaurant,African Restaurant,American Restaurant,French Restaurant,Public Art,Cosmetics Shop,Seafood Restaurant,Bookstore,Beer Bar,Café,Gym,Tapas Restaurant,Ethiopian Restaurant,Event Space,Gym / Fitness Center,Boutique,Library,Pizza Place,Caribbean Restaurant,Fried Chicken Joint,Spa,Southern / Soul Food Restaurant,Music Venue,Bar,Cafeteria,Park,History Museum,Dessert Shop,Cycle Studio,Jazz Club,Art Gallery,Juice Bar,Bagel Shop,BBQ Joint,Electronics Store,English Restaurant,Design Studio,Food & Drink Shop,Duty-free Shop,Dim Sum Restaurant,Flower Shop,Diner,Dry Cleaner,Flea Market,Fish Market,Eastern European Restaurant,Filipino Restaurant,Discount Store,Fast Food Restaurant,Dumpling Restaurant
3,Chelsea,Italian Restaurant,Coffee Shop,Bakery,Ice Cream Shop,Nightclub,Seafood Restaurant,American Restaurant,Hotel,Art Gallery,Theater,Café,Mexican Restaurant,Tapas Restaurant,Asian Restaurant,Gift Shop,Bookstore,Cupcake Shop,Cycle Studio,Men's Store,Burger Joint,Breakfast Spot,French Restaurant,Butcher,Shoe Repair,Speakeasy,Flea Market,Park,Fish Market,Scenic Lookout,Sandwich Place,Electronics Store,Dessert Shop,Steakhouse,College Theater,New American Restaurant,Noodle House,Poke Place,Office,Pizza Place,Coworking Space,Creperie,Pet Store,Cocktail Bar,Market,Paella Restaurant,Gym / Fitness Center,Israeli Restaurant,Grocery Store,Indie Theater,Trail
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Ice

In [43]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 0, 4, 0, 3], dtype=int32)

In [44]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Discount Store,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Bank,Deli / Bodega,Department Store,Pharmacy,Diner,Donut Shop,Gym,Tennis Stadium,Ice Cream Shop,Video Game Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,English Restaurant,Austrian Restaurant,Arepa Restaurant,Eastern European Restaurant,Arcade,Antique Shop,Design Studio,Animal Shelter,Dessert Shop,Dim Sum Restaurant,American Restaurant,African Restaurant,Dive Bar,Dog Run
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Ice Cream Shop,Noodle House,Bakery,Dumpling Restaurant,Salon / Barbershop,Hotpot Restaurant,Cocktail Bar,Coffee Shop,Furniture / Home Store,Malay Restaurant,Greek Restaurant,Spa,Sandwich Place,Bar,Mexican Restaurant,Asian Restaurant,Museum,Music Venue,Nail Salon,Yoga Studio,Paper / Office Supplies Store,New American Restaurant,Optical Shop,Organic Grocery,Pharmacy,Pizza Place,Roof Deck,Shoe Store,Massage Studio,Gym,Jewelry Store,Dessert Shop,Austrian Restaurant,Bike Shop,Bookstore,Boutique,Café,Cosmetics Shop,English Restaurant,Japanese Restaurant,Garden Center,General Entertainment,Grocery Store,Historic Site,Hotel
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Mobile Phone Shop,Bakery,Sandwich Place,Grocery Store,Deli / Bodega,Tapas Restaurant,Lounge,Liquor Store,Latin American Restaurant,Shoe Store,Park,Gym,Donut Shop,Pizza Place,Mexican Restaurant,Supermarket,Wine Shop,Chinese Restaurant,Caribbean Restaurant,Rest Area,Seafood Restaurant,Scenic Lookout,Restaurant,New American Restaurant,Rental Car Location,Paper / Office Supplies Store,Plaza,Pet Store,Pharmacy,Ramen Restaurant,Pool,Italian Restaurant,Women's Store,Market,Kids Store,American Restaurant,Arepa Restaurant,Bar,Beer Bar,Breakfast Spot,Burger Joint,Clothing Store,Cocktail Bar,Coffee Shop,Department Store,Diner,Electronics Store,Frozen Yogurt Shop,Gym / Fitness Center
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Pizza Place,Wine Bar,Bakery,American Restaurant,Spanish Restaurant,Deli / Bodega,Frozen Yogurt Shop,Chinese Restaurant,Park,Bus Station,Caribbean Restaurant,Seafood Restaurant,Coffee Shop,Restaurant,Playground,Yoga Studio,Diner,Pharmacy,Pet Store,Bistro,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Grocery Store,Latin American Restaurant,Juice Bar,History Museum,Supermarket,Convenience Store,Veterinarian,Bar,Trail,Wine Shop,Beer Garden,Flower Shop

In [53]:
manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Discount Store,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Bank,Deli / Bodega,Department Store,Pharmacy,Diner,Donut Shop,Gym,Tennis Stadium,Ice Cream Shop,Video Game Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,English Restaurant,Austrian Restaurant,Arepa Restaurant,Eastern European Restaurant,Arcade,Antique Shop,Design Studio,Animal Shelter,Dessert Shop,Dim Sum Restaurant,American Restaurant,African Restaurant,Dive Bar,Dog Run
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Ice Cream Shop,Noodle House,Bakery,Dumpling Restaurant,Salon / Barbershop,Hotpot Restaurant,Cocktail Bar,Coffee Shop,Furniture / Home Store,Malay Restaurant,Greek Restaurant,Spa,Sandwich Place,Bar,Mexican Restaurant,Asian Restaurant,Museum,Music Venue,Nail Salon,Yoga Studio,Paper / Office Supplies Store,New American Restaurant,Optical Shop,Organic Grocery,Pharmacy,Pizza Place,Roof Deck,Shoe Store,Massage Studio,Gym,Jewelry Store,Dessert Shop,Austrian Restaurant,Bike Shop,Bookstore,Boutique,Café,Cosmetics Shop,English Restaurant,Japanese Restaurant,Garden Center,General Entertainment,Grocery Store,Historic Site,Hotel
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Mobile Phone Shop,Bakery,Sandwich Place,Grocery Store,Deli / Bodega,Tapas Restaurant,Lounge,Liquor Store,Latin American Restaurant,Shoe Store,Park,Gym,Donut Shop,Pizza Place,Mexican Restaurant,Supermarket,Wine Shop,Chinese Restaurant,Caribbean Restaurant,Rest Area,Seafood Restaurant,Scenic Lookout,Restaurant,New American Restaurant,Rental Car Location,Paper / Office Supplies Store,Plaza,Pet Store,Pharmacy,Ramen Restaurant,Pool,Italian Restaurant,Women's Store,Market,Kids Store,American Restaurant,Arepa Restaurant,Bar,Beer Bar,Breakfast Spot,Burger Joint,Clothing Store,Cocktail Bar,Coffee Shop,Department Store,Diner,Electronics Store,Frozen Yogurt Shop,Gym / Fitness Center
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Pizza Place,Wine Bar,Bakery,American Restaurant,Spanish Restaurant,Deli / Bodega,Frozen Yogurt Shop,Chinese Restaurant,Park,Bus Station,Caribbean Restaurant,Seafood Restaurant,Coffee Shop,Restaurant,Playground,Yoga Studio,Diner,Pharmacy,Pet Store,Bistro,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Grocery Store,Latin American Restaurant,Juice Bar,History Museum,Supermarket,Convenience Store,Veterinarian,Bar,Trail,Wine Shop,Beer Garden,Flower Shop

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Pizza Place,Deli / Bodega,Thai Restaurant,Bank,Grocery Store,Street Art,Coffee Shop,Donut Shop,Liquor Store,French Restaurant,Beer Bar,Clothing Store,Sandwich Place,Seafood Restaurant,Cocktail Bar,Gym,Steakhouse,Pet Store,Dance Studio,Spa,Cuban Restaurant,Pharmacy,Restaurant,Historic Site,Convenience Store,Spanish Restaurant,Café,Taco Place,Farmers Market,Falafel Restaurant,Event Space,Exhibit,Ethiopian Restaurant,English Restaurant,Electronics Store,Yoga Studio,Eastern European Restaurant,Duty-free Shop,Dumpling Restaurant,Dry Cleaner,Drugstore,Filipino Restaurant,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant
13,Lincoln Square,Gym / Fitness Center,Plaza,Concert Hall,Theater,French Restaurant,Italian Restaurant,Café,Bakery,Indie Movie Theater,Opera House,Performing Arts Venue,Coffee Shop,Cosmetics Shop,Cycle Studio,Gym,American Restaurant,Hotel,Mediterranean Restaurant,Movie Theater,Park,Furniture / Home Store,Yoga Studio,Wine Shop,Library,Steakhouse,Bar,Playground,Chinese Restaurant,Spa,Dog Run,Fountain,Food Truck,Accessories Store,Bagel Shop,Recreation Center,Seafood Restaurant,School,Farmers Market,Electronics Store,Mexican Restaurant,Salad Place,Burger Joint,Wine Bar,Sporting Goods Shop,Roof Deck,Gift Shop,College Bookstore,Ice Cream Shop,Pet Store,Athletics & Sports
15,Midtown,Coffee Shop,Hotel,Clothing Store,Steakhouse,Theater,Sandwich Place,Spa,Bakery,Cocktail Bar,Sporting Goods Shop,Park,Food Truck,Bookstore,American Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Cycle Studio,Chinese Restaurant,Gym,French Restaurant,Women's Store,Men's Store,Boxing Gym,Brazilian Restaurant,Mediterranean Restaurant,Yoga Studio,Burrito Place,Szechuan Restaurant,Miscellaneous Shop,Smoke Shop,Event Space,Chocolate Shop,Salad Place,Fast Food Restaurant,Sushi Restaurant,Ramen Restaurant,Greek Restaurant,Theme Park Ride / Attraction,Deli / Bodega,Indian Restaurant,Art Gallery,Hawaiian Restaurant,Pilates Studio,Pizza Place,Cuban Restaurant,Gym / Fitness Center,Juice Bar,Grocery Store,Tailor Shop
18,Greenwich Village,Italian Restaurant,Clothing Store,French Restaurant,Sushi Restaurant,Seafood Restaurant,Bakery,Indian Restaurant,Café,Dessert Shop,Sandwich Place,Gourmet Shop,Electronics Store,Coffee Shop,Cosmetics Shop,Cocktail Bar,Caribbean Restaurant,Burger Joint,Boutique,American Restaurant,Spa,Yoga Studio,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Optical Shop,Bookstore,Beer Bar,Jazz Club,Gaming Cafe,Pizza Place,Playground,Snack Place,Chinese Restaurant,Shoe Store,Bagel Shop,Wine Bar,Lebanese Restaurant,Lingerie Store,Record Shop,Watch Shop,Eastern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Mediterranean Restaurant,Bar,Pilates Studio,Tapas Restaurant,New American Restaurant,Cuban Restaurant,Creperie,

In [50]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Marble Hill,Discount Store,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Bank,Deli / Bodega,Department Store,Pharmacy,Diner,Donut Shop,Gym,Tennis Stadium,Ice Cream Shop,Video Game Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,English Restaurant,Austrian Restaurant,Arepa Restaurant,Eastern European Restaurant,Arcade,Antique Shop,Design Studio,Animal Shelter,Dessert Shop,Dim Sum Restaurant,American Restaurant,African Restaurant,Dive Bar,Dog Run
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Sushi Restaurant,Pizza Place,Japanese Restaurant,Deli / Bodega,Ice Cream Shop,Wine Shop,Spa,Mexican Restaurant,Thai Restaurant,Bagel Shop,Sandwich Place,Gym / Fitness Center,Wine Bar,Diner,Park,Pub,Vietnamese Restaurant,Restaurant,Lounge,Sports Bar,Peruvian Restaurant,French Restaurant,Dessert Shop,Mediterranean Restaurant,Burger Joint,Butcher,Café,New American Restaurant,Pool,Monument / Landmark,Dog Run,Farmers Market,Pharmacy,Salon / Barbershop,Bistro,Donut Shop,Nail Salon,Liquor Store,Daycare,Athletics & Sports,Hot Dog Joint,Indian Restaurant,Bakery,Gourmet Shop,Video Store,Health & Beauty Service
14,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Coffee Shop,Wine Shop,Gym / Fitness Center,Gym,French Restaurant,Seafood Restaurant,Sandwich Place,Spa,New American Restaurant,Mediterranean Restaurant,Dog Run,Wine Bar,Indie Theater,Fried Chicken Joint,Cocktail Bar,Poke Place,Comedy Club,Pub,Ice Cream Shop,Residential Building (Apartment / Condo),Roof Deck,Pie Shop,Chinese Restaurant,Caribbean Restaurant,Hotel Bar,Café,Burger Joint,Pizza Place,Peruvian Restaurant,Lounge,Performing Arts Venue,Park,Deli / Bodega,Diner,Dive Bar,Noodle House,Gift Shop,Juice Bar,Music School,Movie Theater,Ethiopian Restaurant,Frozen Yogurt Shop,Food Court,Market,Building,Health Food Store
23,Soho,Clothing Store,Shoe Store,Women's Store,Boutique,Men's Store,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Art Gallery,Café,Design Studio,Dessert Shop,Spa,Furniture / Home Store,Hotel,Bakery,Lingerie Store,Salon / Barbershop,Seafood Restaurant,French Restaurant,Yoga Studio,Vegetarian / Vegan Restaurant,Pilates Studio,Bookstore,Gift Shop,Supermarket,Watch Shop,Mexican Restaurant,Wine Bar,Music Store,Sporting Goods Shop,Swiss Restaurant,Pizza Place,Spanish Restaurant,Japanese Restaurant,Miscellaneous Shop,Karaoke Bar,Candy Store,Hotel Bar,Electronics Store,Dance Studio,Martial Arts Dojo,Optical Shop,Cupcake Shop,American Restaurant,Cycle Studio,History Museum,Arts & Crafts Store,Salad Place,Gourmet Shop
25,Manhattan Valley,Bar,Coffee Shop

In [51]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Spa,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore,Bar,Wine Shop,Thai Restaurant,Cocktail Bar,Grocery Store,Gym / Fitness Center,Bakery,Art Museum,Sports Bar,Italian Restaurant,Hot Dog Joint,Pub,Nail Salon,Vietnamese Restaurant,Playground,Clothing Store,Karaoke Bar,Museum,Kosher Restaurant,Chinese Restaurant,Food Truck,Flower Shop,Bagel Shop,Ramen Restaurant,Shoe Store,Filipino Restaurant,Breakfast Spot,Exhibit,Burger Joint,Mediterranean Restaurant,Salon / Barbershop,Restaurant,Wine Bar,Mexican Restaurant,Women's Store,Ice Cream Shop,Dance Studio,American Restaurant,Vegetarian / Vegan Restaurant,Concert Hall


In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
22,Little Italy,Bakery,Café,Chinese Restaurant,Yoga Studio,Seafood Restaurant,Bubble Tea Shop,Cocktail Bar,Ice Cream Shop,Salon / Barbershop,Sandwich Place,Italian Restaurant,Boutique,Coffee Shop,Japanese Restaurant,Women's Store,French Restaurant,Mediterranean Restaurant,Asian Restaurant,Wine Bar,Clothing Store,Thai Restaurant,Furniture / Home Store,Massage Studio,Speakeasy,Design Studio,Spanish Restaurant,Spa,Dessert Shop,Cosmetics Shop,Snack Place,Breakfast Spot,Flea Market,Martial Arts Dojo,Pizza Place,Chocolate Shop,Vietnamese Restaurant,Salad Place,Falafel Restaurant,Mexican Restaurant,Karaoke Bar,Rock Club,Newsstand,Pilates Studio,Middle Eastern Restaurant,Miscellaneous Shop,Noodle House,Cycle Studio,Gift Shop,Jewelry Store,Art Gallery
